# CASE,EXTRACT,CAST,substr,入れ子SELECT
### 練習項目
***
- CASEによりコード値をラベルに置き換える
- EXTRACTによりタイムスタンプ型データから年,月,日,時を取り出す
- CASTによりtimestamp型をstring型にキャストする
- substrにより指定文字列を取得する
- 入れ子のSELECT文をFROMの対象にする

### 参考文献
***
- [SQL第2版ゼロからはじめるデータベース操作](https://www.shoeisha.co.jp/book/detail/9784798144450)
- [ビッグデータ分析・活用のためのSQLレシピ](https://book.mynavi.jp/supportsite/detail/9784839961268.html)

In [4]:
%reload_ext google.cloud.bigquery

In [5]:
import pandas as pd
import datetime

## テーブル定義

In [6]:
p_id = 'tactile-acrobat-231812'
name = 'example.mst_users'
data = [
    ('U001', datetime.datetime(2016, 8, 26), 1),
    ('U002', datetime.datetime(2016, 8, 26), 2),
    ('U003', datetime.datetime(2016, 8, 27), 3)
]
cols = ['user_id', 'register_date', 'register_device']

### CREATE

In [7]:
df_create = pd.DataFrame(data, columns=cols)
df_create.to_gbq(name, p_id)

1it [00:05,  5.10s/it]


### UPDATE

In [29]:
%%bigquery
UPDATE `tactile-acrobat-231812.example.mst_users` 
    set amount=100
WHERE
    dt = 'A'

""


### INSERT

In [42]:
%%bigquery
INSERT INTO `tactile-acrobat-231812.example.mst_users` 
    (dt, amount, agg_amount, year_avg_amount)
VALUES
    ('A',0,1,2),
    ('B',1,2,3)

""


### DELETE

In [39]:
%%bigquery
DELETE FROM `tactile-acrobat-231812.example.mst_users`
WHERE
    dt = '2015-01'

""


### DROP TABLE

In [33]:
%%bigquery
DROP TABLE `tactile-acrobat-231812.example.mst_users`

""


### SELECT（*）

In [15]:
%%bigquery df_ast
SELECT
    *
FROM 
  `tactile-acrobat-231812.example.mst_users`

In [16]:
display(df_ast)

,user_id,register_date,register_device
0,U001,2016-08-26 00:00:00+00:00,1
1,U002,2016-08-26 00:00:00+00:00,2
2,U003,2016-08-27 00:00:00+00:00,3


### コード値をラベルに置き換える

In [21]:
%%bigquery df_case
SELECT
    user_id,
    CASE
        WHEN register_device = 1 THEN 'PC'
        WHEN register_device = 2 THEN 'SP'
        WHEN register_device = 3 THEN 'アプリ'
        ELSE 'なし'
    END AS device_name
FROM 
  `tactile-acrobat-231812.example.mst_users`

In [22]:
display(df_case)

,user_id,device_name
0,U001,PC
1,U002,SP
2,U003,アプリ


### タイムスタンプ型データから年,月,日,時を取り出す & 入れ子のSELECT文をFROMの対象にする

In [27]:
%%bigquery df_extract
SELECT
    register_date,
    EXTRACT(YEAR FROM register_date) AS year,
    EXTRACT(MONTH FROM register_date) AS month,
    EXTRACT(DAY FROM register_date) AS day,
    EXTRACT(HOUR FROM register_date) AS hour,
    substr(CAST(register_date AS string), 1, 7) AS year_month
FROM 
    (
    SELECT
        register_date
    FROM
        `tactile-acrobat-231812.example.mst_users`
    WHERE
        register_date = '2016-08-27 00:00:00+00:00'
    ) AS t

In [28]:
display(df_extract)

,register_date,year,month,day,hour,year_month
0,2016-08-27 00:00:00+00:00,2016,8,27,0,2016-08
